# Tugas Besar TF4012 Pengukuran Berbasis Citra
### Real-Time Data Testing

**Nama**: Febricetta Zahraketzia Sarwono </br>
**NIM**: 13319080

In [1]:
# import library
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

# load trained model
savedModel=load_model('signdecoderv2.h5')
savedModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 24,639,365
Trainable params: 1,051,653
Non-trainable params: 23,587,712
_________________________________________________________________


In [2]:
# image size
batch_size = 128
img_height = 299
img_width = 299

In [3]:
# testing function
def sign_predict(model,frame):
    img = tf.image.resize(frame,[img_height,img_width])
    img = tf.image.flip_left_right(img)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    perc = 100*score
    return predictions, perc

In [4]:
# real-time data testing
import cv2 
import numpy as np

# Setup capture
cap = cv2.VideoCapture(0)

while True: 
    ret, frame = cap.read()

    # ROI
    x1 = 20
    y1 = 100

    x2 = 270
    y2 = 375

    # Testing Area
    cv2.rectangle(frame, (x1,y1), (x2,y2), (255,0,0), 2)
    cv2.putText(frame, 'Put your right hand inside the box', (x1,y1-15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

    roi = frame[y1:y2, x1:x2]

    pred, perc = sign_predict(savedModel,roi)

    classified = np.argmax(pred)
    if classified == 0:
        hasil = 'A'
    elif classified == 1:
        hasil = 'C'
    elif classified == 2:
        hasil = 'I'
    elif classified == 3:
        hasil = 'R'
    elif classified == 4:
        hasil = 'T'

    # prediction
    cv2.rectangle(frame, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(frame, hasil, (320,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Sign Language Decoder (SIBI)', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break